### Exponenciación rápida con modulo
(a ** b) mod n\
Algoritmo basado en la respuesta a [esta pregunta de StackOverflow](https://stackoverflow.com/questions/40578553/fast-modular-exponentiation-help-me-find-the-mistake)

In [1]:
def exp_mod(a: int, b: int, n: int) -> int:
    t = 1
    while b:
        t = (t * (a if b%2 else 1)) % n
        a = (a * a) % n
        b >>= 1
    return t

### Test de Miller-Rabin
True si `n` es primo, False otherwise\
Prob. de error menor o igual a 2 ** (-k)\
Algoritmo adaptado de las notas de clases, [Wikipedia](https://es.wikipedia.org/wiki/Test_de_primalidad_de_Miller-Rabin) y [Geeks4Geeks](https://www.geeksforgeeks.org/primality-test-set-3-miller-rabin/)

In [2]:
def miller_rabin(n: int, k: int) -> bool:
    # Casos borde
    if n == 1:
        return False
    if 2 <= n <= 3:
        return True
    d = n - 1;
    s = 0
    while (d % 2 == 0):
        d >>= 1
        s += 1
    for _ in range(k):
        a = random.randint(2, n-2)
        x = exp_mod(a, d, n)
        if x != 1 and x != n-1:
            for i in range(s):
                x = (x * x) % n;
                if (x == 1):
                    return False;
                if (x == n - 1):
                    break
            else:
                return False
    # si pasó las k pruebas
    return True

### Generación de primos
Genera un primo con al menos `l` digitos. Prob de error menor o igual a 2 ** (-100)

In [3]:
def generar_primo(l: int) -> int:
    base = 10 ** l
    techo = base * 10
    seen = set()
    while True:
        candidato = random.randint(base, techo-1)
        if candidato not in seen and miller_rabin(candidato, 100):
            return candidato
        seen.add(candidato)

### Algoritmo Extendido de Euclides
Retorna (MCD(a,b), s, t) tal que MCD(a,b) = s\*a + t\*b

Obtenido de las [notas de clases](https://github.com/UC-IIC3253/2021/blob/main/notas%20de%20clases/05-27.pdf) y de [Wikipedia](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm)

In [4]:
def alg_ext_euclides(a: int, b: int) -> (int, int,int):
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r:
        q = old_r // r
        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t
    return old_r, old_s, old_t


### Inverso modular
`a` y `n` son primos relativos

In [5]:
def inverso(a: int, n: int) -> int:
    _, s, _ = alg_ext_euclides(a, n)
    while s < 0:
        s += n
    return s

### Generación de claves
`d`, `e` y `N` deben ser de largo al menos `l`\
Las claves quedan almacenadas en `public_key.txt` y `private_key.txt`\
Clave publica es `(e,N)`\
Clave privada es `(d,N)`

In [10]:
import random

def generar_clave(l: int) -> None:
    _l = l // 2 + 1
    P = generar_primo(_l)
    Q = generar_primo(_l)
    while Q == P:
        Q = generar_primo(_l)
    N = P * Q
    phi_N = (P-1) * (Q-1)
    rel_primes = False
    while not rel_primes:
        d = random.randint(max(P, Q), phi_N)
        mcd, _, _ = alg_ext_euclides(d, phi_N)
        rel_primes = mcd == 1
    e = inverso(d, phi_N)
    public = (e, N)
    private = (d, N)
    with open('public_key.txt', 'w') as f:
        f.write('\n'.join(str(i) for i in public))
    with open('private_key.txt', 'w') as f:
        f.write('\n'.join(str(i) for i in private))



### RSA: funciones `enc(m)` y `dec(c)`
Funciones de encriptación y decriptación que usan las claves contenidas en los archivos.

In [7]:
def enc(m: int) -> int:
    with open('public_key.txt') as f:
        e, N = (int(i) for i in f.readlines())
        c = exp_mod(m, e, N)
    return c

def dec(m: int) -> int:
    with open('private_key.txt') as f:
        d, N = (int(i) for i in f.readlines())
        c = exp_mod(m, d, N)
    return c